In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
# Create empty lists to hold table data to be scraped
away_box_score = []
home_box_score = []
box_score = []

# input URL and use BeautifulSoup to parse through the page
url = f'https://www.basketball-reference.com/boxscores/201811170ORL.html'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')

# Grab the table element that has away team statistics
away_table = soup.select_one('#div_box-ORL-game-basic') 

# Grab data from table and put it into the list created above
for tr in away_table.select('tr:has(td)'):
    tds = [td.get_text(strip=True) for td in tr.select('td')]
    away_box_score.append(tds)

# Grab the table element that has home team statistics
home_table = soup.select_one('#div_box-LAL-game-basic') 

# Grab data from table and put it into the list created above
for tr in home_table.select('tr:has(td)'):
    tds2 = [td.get_text(strip=True) for td in tr.select('td')]
    home_box_score.append(tds2)

# Grab the table element that has game box score
box_score_table = soup.select_one('#all_line_score')

# Game box score is commented out in html, so this will grab it out of the comments
for comment in box_score_table.find_all(text=lambda text: isinstance(text, Comment)):
    if comment.find("<table ") > 0:
        comment_soup = BeautifulSoup(comment, 'html.parser')
        table = comment_soup.find("table")

# Grab data from table and put it into the list created above
for tr in table.select('tr:has(td)'):
    tds3 = [td.get_text(strip=True) for td in tr.select('td')]
    box_score.append(tds3)


In [3]:
# Create dataframe for away team statistics
away_team_df = pd.DataFrame(away_box_score)

# Create an empty list to store away team statistics header information
away_header_list = []

# Grab the table header information to use as column headers in our away team statistics dataframe
for tr in away_table.select('tr:has(th)'):
    ths = [th.get_text(strip=True) for th in tr.select('th')]
    away_header_list.append(ths)
    
# Create dataframe for home team statistics
home_team_df = pd.DataFrame(home_box_score)

# Create an empty list to store home team statistics header information
home_header_list = []

# Grab the table header information to use as column headers in our home team statistics dataframe
for tr in home_table.select('tr:has(th)'):
    ths2 = [th.get_text(strip=True) for th in tr.select('th')]
    home_header_list.append(ths2)
    
# Create dataframe for game box score
box_score_df = pd.DataFrame(box_score)

# Create an empty list to store game box score dataframe header information
box_score_list = []

# Grab the table header information to use as column headers in our game box score dataframe
for tr in box_score_table.select('tr:has(th)'):
    ths3 = [th.get_text(strip=True) for th in tr.select('th')]
    box_score_list.append(ths3)

In [4]:
# Update the column and row labels
box_score_df = box_score_df.set_axis(['1Q', '2Q', '3Q', '4Q', 'T'], axis=1, inplace=False)
box_score_df = box_score_df.set_axis(['Away', 'Home'])

In [5]:
# Pull the away team player data and combine the lists into one list
away_starters = away_header_list[2:7]
away_bench = away_header_list[8:-1]
away_players = [away_starters,away_bench]

# Pull the home team player data and combine the lists into one list
home_starters = home_header_list[2:7]
home_bench = home_header_list[8:-1]
home_players = [home_starters,home_bench]

In [6]:
# Create a dataframe from away team stats
away_stats_df = pd.DataFrame(away_box_score)

# Create a dataframe from home team stats
home_stats_df = pd.DataFrame(home_box_score)

In [7]:
# Consolodate the away team list of lists into one list
away_starters_list = [item for sublist in away_starters for item in sublist]
away_bench_list = [item for sublist in away_bench for item in sublist]
away_roster = away_starters_list + away_bench_list

# Consolodate the home team list of lists into one list
home_starters_list = [item for sublist in home_starters for item in sublist]
home_bench_list = [item for sublist in home_bench for item in sublist]
home_roster = home_starters_list + home_bench_list

In [8]:
# Add Team Totals to the end of the away team list
away_roster.append('Team Totals')
away_stats_df.index = away_roster

# Add Team Totals to the end of the home team list
home_roster.append('Team Totals')
home_stats_df.index = home_roster

In [9]:
# Set the home and away dataframe column labels
column_headers = away_header_list[1]
column_headers.remove("Starters")
away_stats_df.columns = column_headers
home_stats_df.columns = column_headers

In [10]:
away_stats_df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Evan Fournier,33:31,6,15,.400,1,8,.125,2,3,.667,1,4,5,4,0,0,0,4,15,+8
Nikola Vučević,31:22,15,23,.652,2,5,.400,4,5,.800,4,9,13,0,2,0,3,1,36,+12
Aaron Gordon,29:22,5,13,.385,1,2,.500,1,2,.500,3,3,6,3,2,0,1,1,12,+11
D.J. Augustin,27:07,7,9,.778,3,3,1.000,5,5,1.000,0,1,1,7,0,0,2,2,22,+9
Wes Iwundu,20:03,0,2,.000,0,2,.000,0,0,,0,0,0,1,0,2,1,3,0,+4
Jonathon Simmons,26:03,5,10,.500,0,2,.000,2,3,.667,1,4,5,6,0,0,0,2,12,+9
Jerian Grant,19:14,1,4,.250,1,3,.333,0,0,,1,1,2,3,1,0,1,0,3,+2
Jonathan Isaac,16:54,3,5,.600,1,2,.500,0,0,,1,4,5,1,0,5,2,1,7,+4
Terrence Ross,16:18,4,5,.800,2,2,1.000,3,3,1.000,0,2,2,0,0,0,2,5,13,+9
Mo Bamba,14:54,3,6,.500,0,3,.000,2,2,1.000,0,1,1,1,0,1,1,3,8,+3


In [11]:
home_stats_df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Brandon Ingram,33:39,7,13,.538,0,2,.000,3,4,.750,0,6,6,7,0,0,0,1,17,-7
Lonzo Ball,28:13,0,5,.000,0,3,.000,0,0,,0,5,5,4,2,1,3,4,0,-9
Kyle Kuzma,27:09,3,10,.300,1,4,.250,1,2,.500,0,6,6,0,0,2,5,2,8,-7
LeBron James,25:53,8,19,.421,3,6,.500,3,4,.750,0,4,4,7,0,1,3,0,22,-11
JaVale McGee,15:55,5,7,.714,0,0,,1,2,.500,3,1,4,0,0,0,1,1,11,-6
Kentavious Caldwell-Pope,28:19,3,9,.333,0,3,.000,4,4,1.000,3,1,4,0,0,0,0,3,10,-6
Lance Stephenson,21:40,6,9,.667,1,3,.333,6,6,1.000,1,5,6,3,2,0,1,1,19,-2
Josh Hart,20:11,5,7,.714,3,4,.750,0,0,,0,2,2,1,0,2,0,2,13,-17
Tyson Chandler,17:21,1,1,1.000,0,0,,0,0,,0,3,3,0,0,0,1,1,2,-12
Svi Mykhailiuk,10:30,2,3,.667,2,3,.667,2,2,1.000,0,0,0,0,0,0,0,1,8,+5


In [12]:
box_score_df

,1Q,2Q,3Q,4Q,T
Away,31,22,28,36,117
Home,25,37,38,30,130


In [13]:
text = soup.get_text()
attendance_string = re.findall(r"Attendance:\s+\d*,\d*", text)
game_time_string = re.findall(r"Time of Game:\s+\d:\d*", text)
new_attendance_string = attendance_string[0].strip('([^a-z]xa)')
new_game_time_string = game_time_string[0].strip('([^a-z]xa)')
attendance_list = list(new_attendance_string)
game_time_list = list(new_game_time_string)
attendance_list.pop(11)
game_time_list.pop(13)

'\xa0'

In [14]:
final_attendance_string = "".join(attendance_list)
final_game_time_sting = "".join(game_time_list)

In [15]:
final_attendance_string

'Attendance:19,249'

In [16]:
final_game_time_sting

'Time of Game:2:10'

In [17]:
x = final_attendance_string.split(":")
y = final_game_time_sting.split(":",1)

In [18]:
attendance_df = pd.DataFrame(x, columns = [x[0]])
attendance_df = attendance_df.drop([0])

In [19]:
game_time_df = pd.DataFrame(y, columns = [y[0]])
game_time_df = game_time_df.drop([0])

In [20]:
game_logistics_df = pd.concat([attendance_df, game_time_df], ignore_index=True, sort=False)
game_logistics_df['Time of Game'] = game_logistics_df['Time of Game'].shift(-1)
game_logistics_df = game_logistics_df.dropna()

In [21]:
divparent = soup.find('div', attrs={'class':'scorebox_meta'})
divparent.text
text = divparent.text
result = re.split(r'\n', text)
almost_done = result[1]
final_result = almost_done.partition("2018")[2]
game_logistics_df = game_logistics_df.assign(Venue=[final_result])
game_logistics_df

,Attendance,Time of Game,Venue
0,"19,249",2:10,"Amway Center, Orlando, Florida"
